# В данной тетрадке:

Train и test предложения выбраны из rustance_back_to_preprocessed.csv - этот файл был получен путем возврата rustance_pred_labels.conllu в первоначальный корпусной вид с метками Stance.

train.conllu (CoBaLD) ипользован после import conllu для MyCorpus

rustance_pred_labels.conllu использован для vocab и id2word. Основан на предобработанном rustance.csv

# **2models concatenation**

Для реализации задумки с двумя моделями - одной для слова, другой для его метки - нам следует:

1.   Загрузить **rustance_back_to_preprocessed.csv** - это просто очищенные тексты с 2-мя столбцами text и stance. Или все же rustance_back_to_csv_concat_deps, если хотим склейку.
2.   Для MyCorpus - CoBaLD-based эмбеддингов - загружаем **cobald_train.conllu**
3.   Для model1 и model2 загрузить **rustance_pred_labels.conllu** - отсюда возьмем по отдельности слова и их метки, а также сделаем склейки








In [1]:
!pip install razdel

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
import razdel
import re
import spacy # для потенциального использования
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler
from torch.nn.utils.rnn import pad_sequence
import torch.optim as optim

import gensim.models
from gensim.test.utils import datapath
from gensim import utils
import gensim
import gensim.downloader as api

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [4]:
import random

In [5]:
def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

In [6]:
set_random_seed(5)

Загружаем датасет по классификации текстов

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/scripts_diploma/RuATD_back_to_csv_concat_deps_misc.csv', sep=';')
data.head(150)
for item in data.head(250):
  print(item)

text
majority_vote


In [ ]:
filtered_df = data[data['majority_vote'] == 'H']

In [ ]:
data.majority_vote.unique()

array(['M', 'H'], dtype=object)

In [ ]:
len(filtered_df)

1719

In [ ]:
filtered_df[-114:]

,text,majority_vote
2300,я:agent вернула:predicate книгу:object в:_ биб...,H
2301,я:agent вернулся:predicate в:_ москву:locative...,H
2304,я:agent встал:predicate около:_ пяти:time,H
2306,я:agent говорил:predicate с:_ мудрым:character...,H
2308,я:object действительно:characteristic счастлив...,H
...,...,...
2497,улучшению:possessor_metaphoric экологической:s...,H
2506,численность:object_situation молодых:_ граждан...,H
2509,я:experiencer думала:predicate со:_ временем:t...,H
2510,я:agent щас:time ела:predicate самый:_ вкусный...,H


In [ ]:
le = LabelEncoder()
data.majority_vote = le.fit_transform(data.majority_vote)

In [ ]:
from sklearn.utils import shuffle, class_weight
yweights = class_weight.compute_class_weight('balanced', classes=np.unique(data.majority_vote), y=data.majority_vote)
yweights = torch.tensor(yweights,dtype=torch.float)

In [ ]:
train_sentences, val_sentences = train_test_split(data[['text', 'majority_vote']], test_size=0.1, stratify=data.majority_vote) # вряд ли и стратификация сильно поможет

Датасет очень несбалансированный

In [ ]:
len(train_sentences)

2265

In [ ]:
train_sentences[:100]

,text,majority_vote
2282,это:ch_reference число:agent_metaphoric устано...,1
1860,с:_ мая:interval_beginning по:_ август:time 96...,0
1294,несмотря:_ на:_ успешное:characteristic наступ...,1
228,был:predicate в:_ хлебзаводе:locative угостили...,1
1167,мы:raising_target ничего:object_situation не:_...,0
...,...,...
72,утреннее:time воздушное:locative наблюдение:ob...,1
1606,по:_ результатам:criterion зрительского:agent ...,1
1864,с:_ особым:characteristic интересом:part_situa...,0
1187,на:_ большее:degree расстояние:locative_finalp...,1


In [ ]:
len(val_sentences)

252

Соберем словарь

In [ ]:
texts = list(data.majority_vote)

In [ ]:
texts[:1000]

[1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,


In [9]:
!pip install conllu

In [10]:
import conllu

В следующих ячейках предлагается код для извлечения слов и меток из CoBaLD.

In [ ]:
# ТОЛЬКО СЛОВО

with open('/content/drive/MyDrive/cobald_train.conllu', 'r', encoding='utf-8') as file:
    data = file.read()

parsed_conllu_sentences = conllu.parse(data)
cobald_sentences_words = []
for sentence in parsed_conllu_sentences:
    cursent = []
    for token in sentence:
        #print('ТОКЕН', token)
        if token['upostag'] != 'PUNCT' and token['upostag']:
            cursent.append(token['form'])
    cobald_sentences_words.append(' '.join(cursent))

for i in range(len(cobald_sentences_words)):
  cobald_sentences_words[i] = cobald_sentences_words[i].lower()

print(cobald_sentences_words[:10])

['наверное', 'на месте погибли 25-летний тато карепов 43-летняя наталья карепова и 60-летняя шура таанани', 'на стене дома полиция обнаружила надпись дикая природа не подчиняется', 'об этом сообщает риа новости со ссылкой на управление информации и общественных связей генпрокуратуры рф', 'на станции комсомольская кольцевой ветки московского метрополитена сегодня днем был обнаружен предмет внешне похожий на взрывное устройство сообщает риа новости', 'на российско-грузинскую границу перебрасываются подразделения северо-кавказского военного округа', 'по данным увд тулы у следователей есть основания полагать что это убийство было заказным', 'он уже подписал соответствующие документы а до среды свои подписи под контрактами должны поставить и руководители российской космической отрасли', 'партнером медиа-магната по съемочной площадке стал лауреат премии оскар роберт дюваль играющий роль генерала ли', 'об этом сообщил председатель республиканского избиркома николай ондар']


In [ ]:
# ТОЛЬКО МЕТКА DEPS

with open('/content/drive/MyDrive/cobald_train.conllu', 'r', encoding='utf-8') as file:
    data = file.read()

parsed_conllu_sentences = conllu.parse(data)
cobald_sentences_deps = []
for sentence in parsed_conllu_sentences:
    cursent = []
    for token in sentence:
        #print('ТОКЕН', token)
        if token['upostag'] != 'PUNCT':
            if token['deprel'] != 'punct':
                if token['deprel'] not in '&@\#(%.+?\)':
                    if not token['deps']:
                        token['deps'] = '_'
                    cursent.append(token['deps'].lower())
    cobald_sentences_deps.append(' '.join(cursent))

for i in range(len(cobald_sentences_deps)):
  cobald_sentences_deps[i] = cobald_sentences_deps[i].lower()

print(cobald_sentences_deps[:10])

['predicate', '_ locative predicate time experiencer name_title time experiencer name_title _ time experiencer name_title', '_ locative whole possessor predicate object _ agent_metaphoric _ predicate', '_ theme predicate agent _ _ part_situation _ object object _ experiencer object_situation set_general locative', '_ locative predicate_noun characteristic object_relation locative whole time time _ predicate object locative characteristic _ sphere correlative predicate addressee _', '_ locative locative_finalpoint predicate object locative sphere locative', '_ parenthetical possessor locative _ possessor predicate object_situation explication _ ch_reference object_situation state state', 'agent time predicate characteristic object _ _ time agent $dislocation _ locative predicate object_situation addition raising_target locative sphere object_situation', 'correlative object_relation _ sphere sphere predicate relative object predicate_noun name_title name_title participlerelativeclause ob

In [ ]:
# ТОЛЬКО МЕТКА MISC

with open('/content/drive/MyDrive/cobald_train.conllu', 'r', encoding='utf-8') as file:
    data = file.read()

parsed_conllu_sentences = conllu.parse(data)
cobald_sentences_misc = []
for sentence in parsed_conllu_sentences:
    cursent = []
    for token in sentence:
        #print('ТОКЕН', token)
        if token['upostag'] != 'PUNCT':
            if token['deprel'] != 'punct':
                if token['deprel'] not in '&@\#(%.+?\)':
                    if not token['misc']:
                        token['misc'] = {'_': ''}
                    cursent.append(list(token['misc'].keys())[0].lower())
    cobald_sentences_misc.append(' '.join(cursent))

for i in range(len(cobald_sentences_misc)):
  cobald_sentences_misc[i] = cobald_sentences_misc[i].lower()

print(cobald_sentences_misc[:10])

['modality', 'preposition place physical_psychic_condition time being being time being _ coordinating_conjunctions time being being', 'preposition part_of_construction construction_as_whole organization to_seek_find text_objects_and_documents ch_by_residence relative_space particles hierarchical_verbs', 'preposition entity_or_situation_pronoun verbal_communication companies information preposition verbal_communication preposition state_authorities information coordinating_conjunctions human interpersonal_relations state_authorities country_as_administrative_unit', 'preposition place urban_space_and_roads ch_configuration_and_form part_or_portion_of_entity inhabited_locality transport time time auxiliary_verbs to_seek_find physical_object ch_disposition_and_motion ch_of_connections preposition to_blow_up device verbal_communication companies information', 'preposition country_as_administrative_unit lines to_send_to_deliver military_forces_as_organization the_earth_and_its_spatial_parts 

In [ ]:
# CoBaLD with Concatenated Labels Deps Misc

with open('/content/drive/MyDrive/cobald_train.conllu', 'r', encoding='utf-8') as file:
    data = file.read()

parsed_cobald_sentences = conllu.parse(data)
cobald_sentences_words_and_deps = []
for sentence in parsed_cobald_sentences:
    cursent = []
    for token in sentence:
        #print('ТОКЕН', token)
        if token['upostag'] != 'PUNCT' and token['upostag']:
            if token['deprel'] != 'punct':
                if token['deprel'] not in '&@\#(%.+?\)':
                    if not token['deps']:
                        token['deps'] = '_'
                    if not token['misc']:
                        token['misc'] = {'_': ''}
                    cursent.append(f"{token['form'].lower()}:{token['deps'].lower()}")
    cobald_sentences_words_and_deps.append(' '.join(cursent))

for i in range(len(cobald_sentences_words_and_deps)):
  cobald_sentences_words_and_deps[i] = cobald_sentences_words_and_deps[i].lower()

cobald_sentences_words_deps_misc = cobald_sentences_words_and_deps[::]
print(cobald_sentences_words_deps_misc[:10])

['наверное:predicate', 'на:_ месте:locative погибли:predicate 25-летний:time тато:experiencer карепов:name_title 43-летняя:time наталья:experiencer карепова:name_title и:_ 60-летняя:time шура:experiencer таанани:name_title', 'на:_ стене:locative дома:whole полиция:possessor обнаружила:predicate надпись:object дикая:_ природа:agent_metaphoric не:_ подчиняется:predicate', 'об:_ этом:theme сообщает:predicate риа:agent новости:_ со:_ ссылкой:part_situation на:_ управление:object информации:object и:_ общественных:experiencer связей:object_situation генпрокуратуры:set_general рф:locative', 'на:_ станции:locative комсомольская:predicate_noun кольцевой:characteristic ветки:object_relation московского:locative метрополитена:whole сегодня:time днем:time был:_ обнаружен:predicate предмет:object внешне:locative похожий:characteristic на:_ взрывное:sphere устройство:correlative сообщает:predicate риа:addressee новости:_', 'на:_ российско-грузинскую:locative границу:locative_finalpoint перебрасываю

In [ ]:
len(cobald_sentences_words)

34414

In [ ]:
len(cobald_sentences_deps)

34414

In [ ]:
len(cobald_sentences_misc)

34414

In [ ]:
len(cobald_sentences_words_deps_misc)

34414

В дальнейших ячейках собираются корпуса и обучается модель.

In [ ]:
class MyCorpusWords:
    """An iterator that yields CoBaLD sentences (lists of str)."""

    def __iter__(self):
        for sent in cobald_sentences_words:
            # в каждом sent - слова через пробел без пунктуации
            yield utils.simple_preprocess(sent)

In [ ]:
corpus_words = MyCorpusWords()
model_words = gensim.models.Word2Vec(sentences=corpus_words)

In [ ]:
class MyCorpusDeps:
    """An iterator that yields CoBaLD sentences (lists of str)."""

    def __iter__(self):
        for sent in cobald_sentences_deps:
            # в каждом sent - слова через пробел без пунктуации
            yield utils.simple_preprocess(sent)

In [ ]:
corpus_deps = MyCorpusDeps()
model_deps = gensim.models.Word2Vec(sentences=corpus_deps, vector_size=50)

In [ ]:
class MyCorpusMisc:
    """An iterator that yields CoBaLD sentences (lists of str)."""

    def __iter__(self):
        for sent in cobald_sentences_misc:
            # в каждом sent - слова через пробел без пунктуации
            yield utils.simple_preprocess(sent)

In [ ]:
corpus_misc = MyCorpusMisc()
model_misc = gensim.models.Word2Vec(sentences=corpus_misc, vector_size=50)

In [ ]:
class MyCorpus3:
    """An iterator that yields CoBaLD sentences (lists of str)."""

    def __iter__(self):
        for sent in cobald_sentences_words_deps_misc:
            # в каждом sent - слова через пробел без пунктуации
            yield utils.simple_preprocess(sent)

In [ ]:
corpus3 = MyCorpus3()
model_3 = gensim.models.Word2Vec(sentences=corpus3, vector_size=120)

Здесь мы должны собрать словарь из датасета, на котором собираемся решать задачу.

In [11]:
# RuATD RuATD Misc

with open('/content/drive/MyDrive/scripts_diploma/RuATD.conllu', 'r', encoding='utf-8') as file:
    data = file.read()

parsed_RuATD_sentences = conllu.parse(data)
RuATD_sentences_words_and_deps = []
for sentence in parsed_RuATD_sentences:
    cursent = []
    for token in sentence:
        #print('ТОКЕН', token)
        if token['upostag'] != 'PUNCT' and token['upostag']:
            if token['deprel'] != 'punct':
                if token['deprel'] not in '&@\#(%.+?\)':
                    if not token['deps']:
                        token['deps'] = '_'
                    if not token['misc']:
                        token['misc'] = {'_': ''}
                    cursent.append(f"{token['form'].lower()}:{token['deps'].lower()}:{list(token['misc'].keys())[0].lower()}")
    RuATD_sentences_words_and_deps.append(' '.join(cursent))

for i in range(len(RuATD_sentences_words_and_deps)):
  RuATD_sentences_words_and_deps[i] = RuATD_sentences_words_and_deps[i].lower()

RuATD_sentences_words_deps_misc = RuATD_sentences_words_and_deps[::]
print(RuATD_sentences_words_deps_misc[:10])

['ваши:object_relation:being мартовские:time:time наши:object_relation:being коты:object:animal весной:time:time', 'прекрасные:characteristic:ch_evaluation новости:predicate_noun:information', 'гомосексуальность:object_situation:physical_and_biological_properties вей:object:being усяня:name_title:being мертва:state:physical_psychic_condition', 'большое:degree:ch_degree искреннее:characteristic:ch_evaluation_of_human_temper_and_activity спасибо:predicate_discoursiveunits:discoursive_units буду:_:auxiliary_verbs знать:predicate:state_of_mind какую:ch_reference:ch_reference_and_quantification кофейню:object:public_service_institutions точно:characteristic:degree_of_fit стоит:object_situation:modality обходить:object_situation:to_avoid стороной:_:ch_disposition_and_motion если:_:conjunctions решу:time:to_decide остаться:object_situation:position_in_space в:_:preposition спб:locative:inhabited_locality', 'как:_:ch_degree красиво:characteristic:ch_evaluation упал:predicate:motion прямо:degre

In [ ]:
# CoBaLD with Concatenated Labels Deps Misc

with open('/content/drive/MyDrive/cobald_train.conllu', 'r', encoding='utf-8') as file:
    data = file.read()

parsed_cobald_sentences = conllu.parse(data)
cobald_sentences_words_and_deps = []
for sentence in parsed_cobald_sentences:
    cursent = []
    for token in sentence:
        #print('ТОКЕН', token)
        if token['upostag'] != 'PUNCT' and token['upostag']:
            if token['deprel'] != 'punct':
                if token['deprel'] not in '&@\#(%.+?\)':
                    if not token['deps']:
                        token['deps'] = '_'
                    if not token['misc']:
                        token['misc'] = {'_': ''}
                    cursent.append(f"{token['form'].lower()}:{token['deps'].lower()}:{list(token['misc'].keys())[0].lower()}")
    cobald_sentences_words_and_deps.append(' '.join(cursent))

for i in range(len(cobald_sentences_words_and_deps)):
  cobald_sentences_words_and_deps[i] = cobald_sentences_words_and_deps[i].lower()

cobald_sentences_words_deps_misc = cobald_sentences_words_and_deps[::]
print(cobald_sentences_words_deps_misc[:10])

['наверное:predicate:modality', 'на:_:preposition месте:locative:place погибли:predicate:physical_psychic_condition 25-летний:time:time тато:experiencer:being карепов:name_title:being 43-летняя:time:time наталья:experiencer:being карепова:name_title:_ и:_:coordinating_conjunctions 60-летняя:time:time шура:experiencer:being таанани:name_title:being', 'на:_:preposition стене:locative:part_of_construction дома:whole:construction_as_whole полиция:possessor:organization обнаружила:predicate:to_seek_find надпись:object:text_objects_and_documents дикая:_:ch_by_residence природа:agent_metaphoric:relative_space не:_:particles подчиняется:predicate:hierarchical_verbs', 'об:_:preposition этом:theme:entity_or_situation_pronoun сообщает:predicate:verbal_communication риа:agent:companies новости:_:information со:_:preposition ссылкой:part_situation:verbal_communication на:_:preposition управление:object:state_authorities информации:object:information и:_:coordinating_conjunctions общественных:experi

In [ ]:
vocab = Counter()
# Для словаря RUSTANCE
for text in RuATD_sentences_words_deps_misc:
    vocab.update(text.split())
print('всего уникальных токенов:', len(vocab))

всего уникальных токенов: 31711


In [ ]:
# Для словаря CoBaLD
for text in cobald_sentences_words_and_deps:
    vocab.update(text.split())
print('всего уникальных токенов:', len(vocab))

всего уникальных токенов: 113344


In [ ]:
filtered_vocab = set()

for word in vocab:
    if vocab[word] > 2:
        filtered_vocab.add(word)
print('уникальных токенов, втретившихся больше 2 раз:', len(filtered_vocab))

уникальных токенов, втретившихся больше 2 раз: 21769


In [ ]:
filtered_vocab

{'последующих:orderintimeandspace',
 'госдолга:object',
 'эффективный:characteristic',
 '103:quantity',
 'национальном:characteristic',
 'распоряжением:instrument',
 'филиалы:object',
 'ленина:object',
 'игрушек:object',
 'городских:characteristic',
 'вернулся:predicate',
 'тяжело:state',
 'воздушное:locative',
 'заголовки:predicate_noun',
 'киевский:locative',
 'производственных:sphere',
 'услуги:object',
 'секретаря:object_relation',
 'органы:agent',
 'венесуэле:locative',
 'газов:whole',
 'переход:object_situation',
 'привело:participlerelativeclause',
 'магазина:whole',
 'просили:predicate',
 'принадлежности:theme',
 'скидка:object_situation',
 'материалы:object',
 'страны:agent',
 'инициировано:predicate',
 'территориях:locative',
 'принцип:relative',
 'стеклянных:fabricative',
 'хозяйствах:locative',
 'протест:object_situation',
 'уведомление:object',
 'войсками:agent',
 'находилось:object_situation',
 'потратить:object_situation',
 'com:name_title',
 'модели:object',
 'долл:stat

In [ ]:
#создаем словарь с индексами word2id, для спецсимвола паддинга дефолтный индекс - 0
word2id = {'PAD': 0}

for word in filtered_vocab:
    word2id[word] = len(word2id)

In [ ]:
word2id

{'PAD': 0,
 'последующих:orderintimeandspace': 1,
 'госдолга:object': 2,
 'эффективный:characteristic': 3,
 '103:quantity': 4,
 'национальном:characteristic': 5,
 'распоряжением:instrument': 6,
 'филиалы:object': 7,
 'ленина:object': 8,
 'игрушек:object': 9,
 'городских:characteristic': 10,
 'вернулся:predicate': 11,
 'тяжело:state': 12,
 'воздушное:locative': 13,
 'заголовки:predicate_noun': 14,
 'киевский:locative': 15,
 'производственных:sphere': 16,
 'услуги:object': 17,
 'секретаря:object_relation': 18,
 'органы:agent': 19,
 'венесуэле:locative': 20,
 'газов:whole': 21,
 'переход:object_situation': 22,
 'привело:participlerelativeclause': 23,
 'магазина:whole': 24,
 'просили:predicate': 25,
 'принадлежности:theme': 26,
 'скидка:object_situation': 27,
 'материалы:object': 28,
 'страны:agent': 29,
 'инициировано:predicate': 30,
 'территориях:locative': 31,
 'принцип:relative': 32,
 'стеклянных:fabricative': 33,
 'хозяйствах:locative': 34,
 'протест:object_situation': 35,
 'уведомлен

In [ ]:
#обратный словарь для того, чтобы раскодировать последовательность
id2word = {i: word for word, i in word2id.items()}

In [ ]:
id2word

{0: 'PAD',
 1: 'последующих:orderintimeandspace',
 2: 'госдолга:object',
 3: 'эффективный:characteristic',
 4: '103:quantity',
 5: 'национальном:characteristic',
 6: 'распоряжением:instrument',
 7: 'филиалы:object',
 8: 'ленина:object',
 9: 'игрушек:object',
 10: 'городских:characteristic',
 11: 'вернулся:predicate',
 12: 'тяжело:state',
 13: 'воздушное:locative',
 14: 'заголовки:predicate_noun',
 15: 'киевский:locative',
 16: 'производственных:sphere',
 17: 'услуги:object',
 18: 'секретаря:object_relation',
 19: 'органы:agent',
 20: 'венесуэле:locative',
 21: 'газов:whole',
 22: 'переход:object_situation',
 23: 'привело:participlerelativeclause',
 24: 'магазина:whole',
 25: 'просили:predicate',
 26: 'принадлежности:theme',
 27: 'скидка:object_situation',
 28: 'материалы:object',
 29: 'страны:agent',
 30: 'инициировано:predicate',
 31: 'территориях:locative',
 32: 'принцип:relative',
 33: 'стеклянных:fabricative',
 34: 'хозяйствах:locative',
 35: 'протест:object_situation',
 36: 'уведо

weights будет наша матрица эмбеддингов для слов из словаря, который мы собрали, взятых из модели w2v. Потом мы инициализируем эмбеддинги в нейронной модельке из этих весов.

In [ ]:
# ПРИМЕР инициализации весов для сконкатенированных векторов слов и классов

weights = np.zeros((len(word2id), 150))
count = 0
for word, i in word2id.items():
    if word == 'PAD':
        continue
    try:
        word_form, word_deps = word.split(':')[0], word.split(':')[1]
    except:
        weights[i] = np.random.normal(0,0.1,150)
        continue

    try:
        word_vector = model_words.wv[word_form]
    except KeyError:
        word_vector = np.random.normal(0,0.1,100)

    try:
        label_vector = model_deps.wv[word_deps]
    except KeyError:
        label_vector = np.random.normal(0,0.1,50)

    weights[i] = np.concatenate((word_vector, label_vector))

In [ ]:
# ПРИМЕР инициализации весов для сконкатенированных векторов слов, классов и ролей

weights = np.zeros((len(word2id), 200))
count = 0
for word, i in word2id.items():
    if word == 'PAD':
        continue
    try:
        word_form, word_deps, word_misc = word.split(':')[0], word.split(':')[1], word.split(':')[2]
    except:
        weights[i] = np.random.normal(0,0.1,200)
        continue
    try:
        word_vector = model_words.wv[word_form]
    except KeyError:
        word_vector = np.random.normal(0,0.1,100)

    try:
        deps_vector = model_deps.wv[word_deps]
    except KeyError:
        deps_vector = np.random.normal(0,0.1,50)

    try:
        misc_vector = model_misc.wv[word_misc]
    except KeyError:
        misc_vector = np.random.normal(0,0.1,50)

    # weights[i] = word_vector + deps_vector + misc_vector
    weights[i] = np.concatenate((word_vector, deps_vector, misc_vector))

In [ ]:
# ПРИМЕР инициализации весов для склейки слов, классов и ролей

weights = np.zeros((len(word2id), 120))
count = 0
for word, i in word2id.items():
    if word == 'PAD':
        continue
    try:
        weights[i] = model_3.wv[word]
    except KeyError:
      count += 1 # это можно потом вывести, чтобы узнать, сколько слов у нас OOV
      # oov словам сопоставляем случайный вектор
      weights[i] = np.random.normal(0,0.1,120)

In [ ]:
count

18719

In [ ]:
weights

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.14928307, -0.13436971,  0.03962008, ...,  0.06399186,
         0.01709341,  0.12712404],
       [ 0.01499895, -0.02104161,  0.12147466, ..., -0.1540363 ,
        -0.00420191,  0.0710262 ],
       ...,
       [ 0.08825474,  0.10727892,  0.06379178, ...,  0.07166782,
         0.00750793,  0.03160035],
       [ 0.18283324, -0.062598  ,  0.09722364, ...,  0.06854567,
        -0.16389911, -0.10069583],
       [-0.11222731,  0.07964428, -0.07011978, ..., -0.13749129,
         0.03135638,  0.2325225 ]])

Готовим датасет и архитектуру простенького многослойного перцептрона в torch. Девайс можно в колабчике выбрать gpu - у меня на ноуте не настроена куда, да и задача не требует мощностей, поэтому у меня далее везде на cpu производились вычисления

In [ ]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
DEVICE

device(type='cpu')

Соберем класс для датасета. У него обязательно должны быть перегружены методы init, len, getitem и желательно должен присутствовать метод collate_fn, который будет обрабатывать данные в батче для передачи в нейронную сеть. Я также вписала метод preprocess для предобработки текстов.

In [ ]:
class StanceDataset(Dataset):

    def __init__(self, dataset, word2id, DEVICE):
        self.dataset = dataset['text'].values
        self.word2id = word2id
        self.length = dataset.shape[0]
        self.target = dataset['majority_vote'].values
        self.device = DEVICE

    def __len__(self): #это обязательный метод, он должен уметь считать длину датасета
        return self.length

    def __getitem__(self, index): #еще один обязательный метод. По индексу возвращает элемент выборки
        tokens = self.preprocess(self.dataset[index]) # токенизируем
        ids = torch.LongTensor([self.word2id[token] for token in tokens if token in self.word2id])
        y = [self.target[index]]
        return ids, y

    def preprocess(self, text):
        t = re.sub(r'@\(.+?\)', '', text)
        tokens = t.lower().split()
        return tokens

    def collate_fn(self, batch): #этот метод можно реализовывать и отдельно,
    # он понадобится для DataLoader во время итерации по батчам
      ids, y = list(zip(*batch))
      padded_ids = pad_sequence(ids, batch_first=True).to(self.device)
      y = torch.Tensor(y).to(self.device) # tuple ([1], [0], [1])  -> Tensor [[1.], [0.], [1.]]
      return padded_ids, y

Напишем архитектуру модели. У нее должно быть два метода - init со слоями и функциями и forward, который неявным образом вызывается в \_\_call\_\_.

In [ ]:
class MLP_w2v(nn.Module):

    def __init__(self, vocab_size):

        super().__init__()
        # указываем в атрибутах класса, какие слои и активации нам понадобятся
        self.embedding = nn.Embedding(vocab_size, 120) # инициализируем эмбеддинги
        self.embedding.from_pretrained(torch.tensor(weights), freeze=True) # возьмем наши вордтувеки и выставим freeze, чтобы их веса не изменялись при обучении
        self.emb2h = nn.Linear(120, 10) # полносвязный скрытый слой со 100 на 10 - эти параметры меняются, первый - в зависимости от длины эмбеддинга
        self.act1 = nn.ReLU() # нелинейная функция активации - обязательно должна быть между двумя слоями
        self.dropout = nn.Dropout(p=0.5) # регуляризация: может и не быть
        self.h2out = nn.Linear(10, 2) # выходной слой: возвращает 4 сырых числа для каждого объекта, т.к. у нас 4 класса (2)


    def forward(self, text): #необходимый метод,  в нем указываем, как именно связываются слои/активации между собой

        embedded = self.embedding(text)   # переводим последовательность индексов в последовательность эмбеддингов
        # считаем средний эмбеддинг предложения - Doc2Vec лучше, но тогда не вполне понятно, как, допустим, конкатенировать эмбеддинги с сем. информацией
        mean_emb = torch.mean(embedded, dim=1)
        hidden = self.emb2h(mean_emb) # пропускаем эмбеддинг через полносвязный слой
        hidden = self.dropout(hidden) # применяем дропаут
        hidden = self.act1(hidden) # функцию активации
        hidden = self.dropout(hidden) # еще раз дропаут
        out = self.h2out(hidden).softmax(dim=1) # возвращаем четыре вероятности классов - по вероятности на класс

        return out

Создадим инстансы датасета для обучения

In [ ]:
train_dataset = StanceDataset(train_sentences, word2id, DEVICE)
train_sampler = RandomSampler(train_dataset)
train_iterator = DataLoader(train_dataset, collate_fn = train_dataset.collate_fn, sampler=train_sampler, batch_size=96)

In [ ]:
val_dataset = StanceDataset(val_sentences, word2id, DEVICE)
val_sampler = RandomSampler(val_dataset)
val_iterator = DataLoader(val_dataset, collate_fn = val_dataset.collate_fn, sampler=val_sampler, batch_size=96)

In [ ]:
batch = next(iter(train_iterator))

In [ ]:
test_batch = next(iter(val_iterator))

In [ ]:
test_batch

(tensor([[13437,  1493,  5784,  ...,     0,     0,     0],
         [ 8574,  9115,  4802,  ...,     0,     0,     0],
         [21694, 17513,  2535,  ...,     0,     0,     0],
         ...,
         [ 8574,  7445,  5961,  ...,     0,     0,     0],
         [10189,  9086,  7947,  ...,     0,     0,     0],
         [15105, 15332,  8574,  ...,     0,     0,     0]]),
 tensor([[0.],
         [0.],
         [0.],
         [0.],
         [1.],
         [0.],
         [0.],
         [0.],
         [1.],
         [1.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [1.],
         [1.],
         [1.],
         [0.],
         [0.],
         [0.],
         [0.],
         [1.],
         [1.],
         [1.],
         [0.],
         [0.],
         [0.],
         [0.],
         [1.],
         [0.],
         [0.],
         [0.],
         [0.],
         [1.],
         [0.],
         [1.],
         [0.],
         [0.],
         [0.],


In [ ]:
batch, y = next(iter(train_iterator))
batch, y = batch.to(device='cpu'), y.to(device='cpu')
print(batch.shape)
print(y.shape)

torch.Size([96, 205])
torch.Size([96, 1])


In [ ]:
#пропустим через модель наш первый батч, чтобы проверить, что все работает
model = MLP_w2v(len(id2word))
output = model(batch)
output.argmax(dim=1)

tensor([1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
        0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1,
        0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1,
        0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0])

Напишем лупы и запустим обучение

In [ ]:
model = MLP_w2v(len(word2id)) # веса модели инициализировались рандомными числами
optimizer = optim.RMSprop(model.parameters(), lr=0.0001) # можно менять на другие оптимизаторы
criterion = torch.nn.CrossEntropyLoss(weight=yweights) # используем кросс-энтропию - она подходит для мультиклассовой классификации

# веса модели и значения лосса храним там же, где и все остальные тензоры
model = model.to(DEVICE)
criterion = criterion.to(DEVICE)

In [ ]:
def train(model, iterator, optimizer, criterion):
    print('Training...')
    epoch_loss = 0 # для подсчета среднего лосса на всех батчах
    model.train()  # ставим модель в обучение, явно указываем, что сейчас надо будет хранить градиенты у всех весов

    for i, (texts, ys) in enumerate(iterator): #итерируемся по батчам
        optimizer.zero_grad()  #обнуляем градиенты
        preds_proba = model(texts) #прогоняем данные через модель
        loss = criterion(preds_proba, ys.squeeze().long()) #считаем значение функции потерь
        loss.backward() #считаем градиенты
        optimizer.step() #обновляем веса
        epoch_loss += loss.item() #сохраняем значение функции потерь

        if not (i + 1) % 2:
            print(f'Train loss: {epoch_loss/i}')

    return epoch_loss / len(iterator) # возвращаем среднее значение функции потерь по всей выборке

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
def evaluate(model, iterator, criterion):
    print("\nValidating...")
    epoch_loss = 0
    model.eval()
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for i, (texts, ys) in enumerate(iterator):
            predictions = model(texts)  # делаем предсказания на тесте
            loss = criterion(predictions, ys.squeeze().long())   # считаем значения функции ошибки для статистики
            epoch_loss += loss.item()
            if not (i + 1) % 2:
              print(f'Val loss: {epoch_loss/i}')

            all_predictions.extend(predictions.argmax(dim=1).tolist())
            all_labels.extend(ys.squeeze().tolist())
    print('We expected ', all_labels)
    print('We got ', all_predictions)
    #######
    output = model(batch)
    p5 = output.argmax(dim=1).tolist()
    e5 = y.tolist()
    k = 0
    for i in range(len(p5)):
        if p5[i] == int(e5[i][0]):
            k += 1
        else:
            x = 0
    print(f"{k}/{len(e5)}")
    #######

    f1 = f1_score(all_labels, all_predictions, average='micro')
    print(f'F1 Score: {f1}')

    return epoch_loss / len(iterator) # возвращаем средний лосс по батчам

Запускаем само обучение, например, на 100 эпохах

In [ ]:
losses = []
losses_eval = []

for i in range(100):
    print(f'\nstarting Epoch {i}')
    epoch_loss = train(model, train_iterator, optimizer, criterion)
    losses.append(epoch_loss)

    epoch_loss_on_test = evaluate(model, val_iterator, criterion)
    losses_eval.append(epoch_loss_on_test)


starting Epoch 0
Training...
Train loss: 1.3823581337928772
Train loss: 0.9209919571876526
Train loss: 0.8304249167442321
Train loss: 0.7913060869489398
Train loss: 0.7694267498122321
Train loss: 0.7556202736767855
Train loss: 0.7455733922811655
Train loss: 0.7382528622945149
Train loss: 0.7332703576368444
Train loss: 0.7288979291915894
Train loss: 0.7257262553487506
Train loss: 0.7227829435597295

Validating...
Val loss: 1.3866433501243591
We expected  [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 